### Цель: привести данные к одному формату

Источник - - Название исходного источника, откуда это наблюдение


Smiles	-- Уникальный идентификатор химического соединения, подробнее тут


Exp. Animal	-- Животное, на котором проводился эксперимент


method of administration -- Способ введения препарата


Metric_1 -- 	Значение метрики в общих единицах измерениях


Metric_1 (a.u)	-- Единицы измерения метрики 1


In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('tox_merged_main.csv')

In [3]:
data.head()

,CAS,SMILES
0,30560-22-6,CCCCCCCCCC(=O)NP(=O)(OC)SC
1,67466-14-2,CCCCCC1=CN=C2C(=C1C)C(=NC(=N2)N)N
2,57648-21-2,C1CN(CCC1N2C3=CC=CC=C3NC2=S)CCCC(=O)C4=CC=C(C=...
3,1165-48-6,CC1=C(OC2=C(C1=O)C=CC(=C2CN(C)C)OC)C3=CC=CC=C3
4,31906-04-4,CC(C)(CCCC1=CCC(CC1)C=O)O


In [4]:
data['SMILES'].nunique()

32244

In [5]:
df = pd.read_csv(r'C:\Users\Vlad\Downloads\tox_merged.csv')

In [6]:
df = df.rename(columns = {'CASRN':'CAS'})
df.head()

,CAS,TOXVAL_TYPE,TOXVAL_NUMERIC,TOXVAL_UNITS,SPECIES_COMMON,EXPOSURE_ROUTE,smiles
0,129318-43-0,LD50,552.00,mg/kg,Rat,oral,C(CC(O)(P(=O)(O)O)P(=O)(O)[O-])CN.[Na+]
1,129927-33-9,LD50,735.00,mg/kg,Rat,oral,CCN(CC)CC#CC(C)(C)OC(=O)C(C1CCCCC1)(C2=CC=CC=C...
2,130209-82-4,LD50,0.05,mg/kg,Rat,oral,CC(C)OC(=O)CCCC=CCC1C(CC(C1CCC(CCC2=CC=CC=C2)O...
3,130365-34-3,LD50,250.00,mg/kg,Rat,oral,CCCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+]...
4,130365-35-4,LD50,250.00,mg/kg,Rat,oral,CCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+](...


In [7]:
new_df = pd.merge(df, data, on = 'CAS', how = 'left')

In [8]:
new_df.drop('smiles', axis=1, inplace=True)

In [9]:
new_df.head()

,CAS,TOXVAL_TYPE,TOXVAL_NUMERIC,TOXVAL_UNITS,SPECIES_COMMON,EXPOSURE_ROUTE,SMILES
0,129318-43-0,LD50,552.00,mg/kg,Rat,oral,C(CC(O)(P(=O)(O)O)P(=O)(O)[O-])CN.[Na+]
1,129927-33-9,LD50,735.00,mg/kg,Rat,oral,CCN(CC)CC#CC(C)(C)OC(=O)C(C1CCCCC1)(C2=CC=CC=C...
2,130209-82-4,LD50,0.05,mg/kg,Rat,oral,CC(C)OC(=O)CCCC=CCC1C(CC(C1CCC(CCC2=CC=CC=C2)O...
3,130365-34-3,LD50,250.00,mg/kg,Rat,oral,CCCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+]...
4,130365-35-4,LD50,250.00,mg/kg,Rat,oral,CCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+](...


In [10]:
LD_50 = new_df[new_df['TOXVAL_TYPE'] == 'LD50'].reset_index()
LD_50 = LD_50.rename(columns = {'TOXVAL_NUMERIC':'LD_50', 'TOXVAL_UNITS': 'LD_50 (a.u)', 'SPECIES_COMMON':'Exp. Animal', 'EXPOSURE_ROUTE':'method of administration', 'TOXVAL_TYPE' : 'LD50', 'SMILES':'smiles'})
LD_50 = LD_50.drop(['LD50', 'CAS'], axis=1)
LD_50.shape

(55971, 6)

In [11]:
new_order = ['smiles', 'Exp. Animal', 'method of administration', 'LD_50', 'LD_50 (a.u)']

LD_50 = LD_50[new_order]

In [12]:
LD_50.head()

,smiles,Exp. Animal,method of administration,LD_50,LD_50 (a.u)
0,C(CC(O)(P(=O)(O)O)P(=O)(O)[O-])CN.[Na+],Rat,oral,552.00,mg/kg
1,CCN(CC)CC#CC(C)(C)OC(=O)C(C1CCCCC1)(C2=CC=CC=C...,Rat,oral,735.00,mg/kg
2,CC(C)OC(=O)CCCC=CCC1C(CC(C1CCC(CCC2=CC=CC=C2)O...,Rat,oral,0.05,mg/kg
3,CCCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+]...,Rat,oral,250.00,mg/kg
4,CCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+](...,Rat,oral,250.00,mg/kg


In [13]:
LD_50.groupby('LD_50 (a.u)')['LD_50 (a.u)'].count().sort_values(ascending = False)

LD_50 (a.u)
mg/kg       55170
mL/kg         765
IU/kg          32
units/kg        4
Name: LD_50 (a.u), dtype: int64

In [14]:
for index, row in LD_50.iterrows():
    if 'mg/kg' in row['LD_50 (a.u)']:
        LD_50.at[index, 'LD_50 (a.u)'] = row['LD_50 (a.u)'].replace('mg/kg', 'mg.kg-1')

In [15]:
LD_50.groupby('LD_50 (a.u)')['LD_50 (a.u)'].count().sort_values(ascending = False)

LD_50 (a.u)
mg.kg-1     55170
mL/kg         765
IU/kg          32
units/kg        4
Name: LD_50 (a.u), dtype: int64

In [16]:
LD_50 = LD_50[~LD_50['LD_50 (a.u)'].isin(['mL/kg', 'IU/kg', 'units/kg'])]

In [17]:
LD_50.groupby('LD_50 (a.u)')['LD_50 (a.u)'].count().sort_values(ascending = False)

LD_50 (a.u)
mg.kg-1    55170
Name: LD_50 (a.u), dtype: int64

In [18]:
LC_50 = new_df[new_df['TOXVAL_TYPE'] == 'LC50'].reset_index()
LC_50 = LC_50.rename(columns = {'TOXVAL_NUMERIC':'LC_50', 'TOXVAL_UNITS': 'LC_50 (a.u)', 'SPECIES_COMMON':'Exp. Animal', 'EXPOSURE_ROUTE':'method of administration', 'TOXVAL_TYPE' : 'LC50', 'SMILES':'smiles'})
LC_50 = LC_50.drop(['LC50', 'CAS'], axis=1)
LC_50.shape

(1983, 6)

In [19]:
new_order_1 = ['smiles', 'Exp. Animal', 'method of administration', 'LC_50', 'LC_50 (a.u)']

LC_50 = LC_50[new_order_1]

In [20]:
LC_50.groupby('LC_50 (a.u)')['LC_50 (a.u)'].count().sort_values(ascending = False)

LC_50 (a.u)
mg/L     1477
mg/m3     495
mg/kg       6
ppm         5
Name: LC_50 (a.u), dtype: int64

In [21]:
LC_50.head()

,smiles,Exp. Animal,method of administration,LC_50,LC_50 (a.u)
0,C=O,Mouse,inhalation,454.0000,mg/L
1,C=O,Rat,inhalation,0.2030,mg/L
2,CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,Mouse,inhalation,0.1045,mg/L
3,CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,Rat,inhalation,0.1045,mg/L
4,CC(C)NCC(C1=CC(=C(C=C1)O)O)O.Cl,Mouse,inhalation,10.0000,mg/L


In [22]:
for index, row in LC_50.iterrows():
    if 'mg/kg' in row['LC_50 (a.u)']:
        LC_50.at[index, 'LC_50 (a.u)'] = row['LC_50 (a.u)'].replace('mg/kg', 'mg.kg-1')
    elif 'mg/L' in row['LC_50 (a.u)']:
        LC_50.at[index, 'LC_50 (a.u)'] = row['LC_50 (a.u)'].replace('mg/L', 'mg.L')
    elif 'ppm' in row['LC_50 (a.u)']:
        LC_50.at[index, 'LC_50 (a.u)'] = row['LC_50 (a.u)'].replace('ppm', 'mg.kg-1')

In [23]:
LC_50 = LC_50[~LC_50['LC_50 (a.u)'].isin(['mg/m3'])]

In [24]:
LC_50.groupby('LC_50 (a.u)')['LC_50 (a.u)'].count().sort_values(ascending = False)

LC_50 (a.u)
mg.L       1477
mg.kg-1      11
Name: LC_50 (a.u), dtype: int64

In [25]:
NOAEL = new_df[new_df['TOXVAL_TYPE'] == 'NOAEL'].reset_index()
NOAEL = NOAEL.rename(columns = {'TOXVAL_NUMERIC':'NOAEL', 'TOXVAL_UNITS': 'NOAEL (a.u)', 'SPECIES_COMMON':'Exp. Animal', 'EXPOSURE_ROUTE':'method of administration', 'TOXVAL_TYPE' : 'NOAEL_', 'SMILES':'smiles'})
NOAEL = NOAEL.drop(['NOAEL_', 'CAS'], axis=1)
NOAEL.shape

(13886, 6)

In [26]:
new_order_1 = ['smiles', 'Exp. Animal', 'method of administration', 'NOAEL', 'NOAEL (a.u)']

NOAEL = NOAEL[new_order_1]

In [27]:
NOAEL.head()

,smiles,Exp. Animal,method of administration,NOAEL,NOAEL (a.u)
0,CCCCOCCO,Mouse,dermal,1000.0,mg/kg-day
1,[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[O-]S(=O)(...,Rat,oral,10265.0,mg/kg-day
2,CCCCCCCC[Sn](=O)CCCCCCCC,Rat,oral,1.8,mg/kg-day
3,COCCO,Mouse,oral,250.0,mg/kg-day
4,[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[O-]S(=O)(...,Rat,oral,10265.0,mg/kg-day


In [28]:
NOAEL.groupby('NOAEL (a.u)')['NOAEL (a.u)'].count().sort_values(ascending = False)

NOAEL (a.u)
mg/kg-day    12353
ppm           1268
mg/m3          149
mg/L           113
mg/kg            3
Name: NOAEL (a.u), dtype: int64

Плотность вещества: 1 ppm = 1 mg/kg

In [29]:
for index, row in NOAEL.iterrows():
    if 'mg/kg-day' in row['NOAEL (a.u)']:
        NOAEL.at[index, 'NOAEL (a.u)'] = row['NOAEL (a.u)'].replace('mg/kg-day', 'mg.kg-1')
    elif 'mg/kg' in row['NOAEL (a.u)']:
        NOAEL.at[index, 'NOAEL (a.u)'] = row['NOAEL (a.u)'].replace('mg/kg', 'mg.kg-1')
    elif 'ppm' in row['NOAEL (a.u)']:
        NOAEL.at[index, 'NOAEL (a.u)'] = row['NOAEL (a.u)'].replace('ppm', 'mg.kg-1')
    elif 'mg/L' in row['NOAEL (a.u)']:
        NOAEL.at[index, 'NOAEL (a.u)'] = row['NOAEL (a.u)'].replace('mg/L', 'mg.L')


In [30]:
NOAEL = NOAEL[~NOAEL['NOAEL (a.u)'].isin(['mg/m3'])]

In [31]:
NOAEL.groupby('NOAEL (a.u)')['NOAEL (a.u)'].count().sort_values(ascending = False)

NOAEL (a.u)
mg.kg-1    13624
mg.L         113
Name: NOAEL (a.u), dtype: int64

In [32]:
LD_50.head()

,smiles,Exp. Animal,method of administration,LD_50,LD_50 (a.u)
0,C(CC(O)(P(=O)(O)O)P(=O)(O)[O-])CN.[Na+],Rat,oral,552.00,mg.kg-1
1,CCN(CC)CC#CC(C)(C)OC(=O)C(C1CCCCC1)(C2=CC=CC=C...,Rat,oral,735.00,mg.kg-1
2,CC(C)OC(=O)CCCC=CCC1C(CC(C1CCC(CCC2=CC=CC=C2)O...,Rat,oral,0.05,mg.kg-1
3,CCCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+]...,Rat,oral,250.00,mg.kg-1
4,CCCCCCCCCCCCCCCNP1(=S)OCC2=C(O1)C=CC(=C2)[N+](...,Rat,oral,250.00,mg.kg-1


In [33]:
LD_50.shape

(55170, 5)

In [34]:
LC_50.head()

,smiles,Exp. Animal,method of administration,LC_50,LC_50 (a.u)
0,C=O,Mouse,inhalation,454.0000,mg.L
1,C=O,Rat,inhalation,0.2030,mg.L
2,CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,Mouse,inhalation,0.1045,mg.L
3,CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,Rat,inhalation,0.1045,mg.L
4,CC(C)NCC(C1=CC(=C(C=C1)O)O)O.Cl,Mouse,inhalation,10.0000,mg.L


In [35]:
NOAEL.head()

,smiles,Exp. Animal,method of administration,NOAEL,NOAEL (a.u)
0,CCCCOCCO,Mouse,dermal,1000.0,mg.kg-1
1,[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[O-]S(=O)(...,Rat,oral,10265.0,mg.kg-1
2,CCCCCCCC[Sn](=O)CCCCCCCC,Rat,oral,1.8,mg.kg-1
3,COCCO,Mouse,oral,250.0,mg.kg-1
4,[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[O-]S(=O)(...,Rat,oral,10265.0,mg.kg-1


In [36]:
df_1 = LC_50.groupby(by=['smiles', 'method of administration', 'Exp. Animal']).agg({'LC_50' : 'mean', 'LC_50 (a.u)': 'first'}).reset_index()
df_1.head()

,smiles,method of administration,Exp. Animal,LC_50,LC_50 (a.u)
0,B(F)(F)F,inhalation,Guinea Pig,0.109,mg.L
1,B(F)(F)F,inhalation,Mouse,1.730,mg.L
2,B(F)(F)F,inhalation,Rat,1.180,mg.L
3,B(F)(F)F.O.O,inhalation,Rat,1.210,mg.L
4,B1=BB=BB=BC=CB=BB=B1,inhalation,Mouse,0.170,mg.L


In [37]:
df_2 = LD_50.groupby(by=['smiles', 'method of administration', 'Exp. Animal']).agg({'LD_50' : 'mean', 'LD_50 (a.u)': 'first'}).reset_index()
df_2.head()

,smiles,method of administration,Exp. Animal,LD_50,LD_50 (a.u)
0,B(=O)OB1OB2OB(OB(OB(O2)OB(O1)OB=O)[O-])[O-].[N...,oral,Guinea Pig,5300.0,mg.kg-1
1,B(=O)OB1OB2OB(OB(OB(O2)OB(O1)OB=O)[O-])[O-].[N...,oral,Rat,2000.0,mg.kg-1
2,B(=O)OB=O,oral,Mouse,3163.0,mg.kg-1
3,B(=O)O[O-].[Na+],oral,Mouse,3250.0,mg.kg-1
4,B(=O)[O-].[Na+],oral,Rat,2330.0,mg.kg-1


In [38]:
df_3 = NOAEL.groupby(by=['smiles', 'method of administration', 'Exp. Animal']).agg({'NOAEL' : 'mean', 'NOAEL (a.u)': 'first'}).reset_index()
df_3.head()

,smiles,method of administration,Exp. Animal,NOAEL,NOAEL (a.u)
0,B(=O)OB1OB2OB(OB(OB(O2)OB(O1)OB=O)[O-])[O-].[N...,oral,Mouse,27.00,mg.kg-1
1,B(=O)OB1OB2OB(OB(OB(O2)OB(O1)OB=O)[O-])[O-].[N...,oral,Rat,82.35,mg.kg-1
2,B(=O)[O-].[Na+],oral,Mouse,27.00,mg.kg-1
3,B(=O)[O-].[Na+],oral,Rat,82.35,mg.kg-1
4,B(CC)(CC)OC,oral,Rat,15.00,mg.kg-1


In [39]:
df_merged = pd.merge(df_1, df_2, on = ['smiles', 'Exp. Animal', 'method of administration'], how = 'outer')

In [40]:
toxval_df = pd.merge(df_merged, df_3, on = ['smiles', 'Exp. Animal', 'method of administration'], how = 'outer')

In [42]:
toxval_df.shape

(44603, 9)

In [44]:
toxval_df.head()

,smiles,method of administration,Exp. Animal,LC_50,LC_50 (a.u),LD_50,LD_50 (a.u),NOAEL,NOAEL (a.u)
0,B(F)(F)F,inhalation,Guinea Pig,0.109,mg.L,NaN,NaN,NaN,NaN
1,B(F)(F)F,inhalation,Mouse,1.730,mg.L,NaN,NaN,NaN,NaN
2,B(F)(F)F,inhalation,Rat,1.180,mg.L,NaN,NaN,NaN,NaN
3,B(F)(F)F.O.O,inhalation,Rat,1.210,mg.L,NaN,NaN,NaN,NaN
4,B1=BB=BB=BC=CB=BB=B1,inhalation,Mouse,0.170,mg.L,NaN,NaN,NaN,NaN


In [43]:
toxval_df.to_csv('toxval_df.csv', index = False)